# Chapter 11 Make it faster, Make it pythonic

Right now our codes work however they do not run very fast. Although the book originally ended on chapter 10 (with extra chapters for certain details such as other design patterns or the situation of C++ quantitative finance in 2008) I have added a new chapter to illustrate how we can put everything together but in a pythonic way, make it run faster.

The first step will be to modify the  MonteCarlo Routine to use vectorized functions with numpy, that work faster.

## Pandas Convergence table

Instead of creating our statistics gatherer and a convergence table we can calculate each statistic using a vectorized operation from numpy and storing the result in pandas dataframe (with the number of paths). If we want to add more statistics we can create a super gatherer that calls other gatherers and stores all the results in a pandas dataframe, joining each dataframe with the number of paths as a key.

In [42]:
from copy import deepcopy
from math import exp, sqrt, log
import pandas as pd

class StatisticsMC:
    def __init__(self):
        pass
    
    def GetResultsSoFar(self):
        pass
    
    def DumpOneResult(self,result,paths):
        pass        
  
    def clone(self):
        pass
    
    def __del__(self):
        pass   
    
class StatisticsMeanConvergence(StatisticsMC):
    def __init__(self):
        self.__DataFrame = pd.DataFrame(columns = ['Number Of Paths', 'Mean'])
        
    def GetResultsSoFar(self): 
        return self.__DataFrame.copy()
    
    def DumpOneResult(self,result,paths):
        mean = np.mean(result)
        self.__DataFrame = self.__DataFrame.append({'Number Of Paths':paths, 'Mean':mean},ignore_index = True)
            
        
    def clone(self):
        return deepcopy(self)
    
    def __del__(self):
        del self

## Fast MonteCarlo

In [23]:
import numpy as np

We modify a little bit the Generator, so we can also ask for numpy random vectors.

In [24]:
class Generator:
    def __init__(self,Dimensionality_ = 1):
        self.__Dimensionality = Dimensionality_
        
    def ResetDimensionality(self,Dimensionality_):
        self.__Dimensionality = Dimensionality_
    
    def GetGaussians(self):
        Variates = np.random.normal(size = self.__Dimensionality)
        return Variates
    
    def GetNGaussians(self,N):
        Variates = np.random.normal(size = N)
        return Variates

Now we add terminators that examine the sample.

In [25]:
import time

class TerminatorMC:
    def __init__(self):
        pass
 
    def GetTerminatorCondition(self,*args,**kwargs):
        pass
    
    def clone(self):
        pass
    
    def __del__(self):
        pass   

class TerminatorRunTime(TerminatorMC):
    def __init__(self,MaxTime_):
        self.__MaxTime = MaxTime_
        self.__TotalTime = 0
        self.__InitTime = -1
    
    def GetTerminatorCondition(self,*args,**kwargs):
        if self.__InitTime == -1:
            self.__InitTime = int(round(time.time() * 1000))
            return False
        else:
            self.__TotalTime += int(round(time.time() * 1000)) - self.__InitTime
            
            if self.__TotalTime >= self.__MaxTime:
                return True
        
            else:
                self.__InitTime = int(round(time.time() * 1000))
                return False
        
    def clone(self):
        return deepcopy(self)
    
    def __del__(self):
        del self
        
class TerminatorMaxPaths(TerminatorMC):
    def __init__(self, MaxPaths_):
        self.__MathPaths = MaxPaths_
 
    def GetTerminatorCondition(self,PayOffs,*args,**kwargs):
        paths = len(PayOffs)
        if paths > self.__MathPaths:
            return True
        else:
            return False
    
    def clone(self):
        return deepcopy(self)
    
    def __del__(self):
        del self  

Now the main MonteCarlo Routine with vectorized operations and numpy arrays.

In [70]:
def SimpleMonteCarlo(TheOption,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths,
                     gatherer,
                     generator,
                    terminator):
    
    Expiry = TheOption.GetExpiry()    
    variance = Vol.IntegralSquare(0,Expiry)
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r.Integral(0,Expiry) + itoCorrection)
    discounting = exp(-r.Integral(0,Expiry))
    vectorized_OptionPayOff = np.vectorize(TheOption.OptionPayOff)
    paths = 2
    thisPayoffs = np.empty(shape = 1)
    thisPayoffs = thisPayoffs[np.logical_not(np.isnan(thisPayoffs))]
    while True:        
        paths = paths**2
        if paths > NumberOfPaths:
            paths = NumberOfPaths
        else:
            pass
        theseGaussians = generator.GetNGaussians(paths)        
        theseSpot = movedSpot*np.exp(rootVariance*theseGaussians)        
        NewPayoffs = vectorized_OptionPayOff(theseSpot)       
        thisPayoffs = np.concatenate((thisPayoffs,NewPayoffs))
        gatherer.DumpOneResult(thisPayoffs*discounting,paths)        
        if terminator.GetTerminatorCondition(thisPayoffs) == True:
            print('Terminator activated, exiting the solver')
            break
        else: 
            pass

Some PayOffs.

In [27]:
class PayOff:
    def __init__(self):
        pass
    
    def __call__(self,Spot):
        pass    

class PayOffBridge:        
    def __del__(self):
        del self
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self = deepcopy(InnerPayOff)
        
class PayOffCall(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)
    
class VanillaOption:
    def __init__(self, ThePayOff_, Expiry_):
        self.__ThePayOff = ThePayOff_
        self.__Expiry = Expiry_
        
    def GetExpiry(self):
        return self.__Expiry
    
    def OptionPayOff(self, Spot):
        return self.__ThePayOff(Spot)

The Parameters classes.

In [32]:
class ParametersInner:
    def __init__(self):
        pass
    
    def Integral(self,time1, time2):
        pass
    
    def IntegralSquare(self,time1,time2):
        pass

class Parameters:    
    def RootMeanSquare(self,time1,time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def Mean(self,time1, time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def copy(self,original):
        from copy import deepcopy
        self = deepcopy(InnerPayOff) 
        
    def __del__(self):
        del self    
        
class ParametersConstant(ParametersInner,Parameters):
    def __init__(self,constant):
        self.__Constant = constant
        self.__ConstantSquare = self.__Constant*self.__Constant
    
    def Integral(self, time1, time2):
        return (time2 - time1)*self.__Constant
    
    def IntegralSquare(self, time1, time2):
        return (time2 - time1)*self.__ConstantSquare  

The Factory.

In [28]:
class PayOffFactory:
    def __init__(self):
        self.__TheCreatorFunctions = {}        
    
    def __del__(self):
        del self
    
    def RegisterPayOff(self, PayOffId, CreatorFunction):
        self.__TheCreatorFunctions[PayOffId] = CreatorFunction
        
    def CreatePayOff(self, PayOffId, *args, **kwargs):
        if PayOffId not in self.__TheCreatorFunctions.keys():
            print(f'{PayOffId} is an unknown payoff')
            return None
        else:
            return self.__TheCreatorFunctions[PayOffId](*args, **kwargs)   

In [29]:
global thePayOffFactory  # We just the global keyword to be accesible from anywhere
thePayOffFactory  = PayOffFactory()
thePayOffFactory.RegisterPayOff("call",PayOffCall)

The Variables.

In [96]:
Strike = 40
Spot = 42
name = "call"
r_ = 0.1
Vol_ = 0.2
NumberOfPaths = 2000000
Maturity = 0.5

We will see how fast this new MonteCarlo runs.

In [97]:
%%time
gatherer = StatisticsMeanConvergence()
generator = Generator()
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
terminator = TerminatorMaxPaths(NumberOfPaths)
if thePayOFF != None:    
    TheOption = VanillaOption(thePayOFF,Maturity)
    r = ParametersConstant(r_)
    Vol = ParametersConstant(Vol_)
    SimpleMonteCarlo(TheOption,Spot,Vol,r,NumberOfPaths, gatherer, generator, terminator)
    results = gatherer.GetResultsSoFar()
    print(results.head(5))    
else:
    print('Incorrect PayOff')

Terminator activated, exiting the solver
   Number Of Paths      Mean
0              4.0  4.176246
1             16.0  7.242969
2            256.0  4.240162
3          65536.0  4.726288
4        2000000.0  4.760205
Wall time: 2.16 s


Now we will compare it with the original routine.

In [84]:
from random import random
def SimpleMonteCarlo1(Expiry,
                     Strike,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths):
    
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry + itoCorrection)
    runningSum = 0
    for i in range(NumberOfPaths):
        thisGaussian = GetOneGaussianByBoxMuller()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = thisSpot - Strike
        if thisPayoff < 0 : 
            thisPayoff = 0
        runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean            

def GetOneGaussianBySummation():
    result = 0
    for j in range(12):
        result += random()
    result -= 6
    return result

def GetOneGaussianByBoxMuller():
    sizeSquared = 1
    while sizeSquared >= 1:
        x = 2*random() - 1
        y = 2*random() - 1
        sizeSquared = x*x + y*y
    
    result = x*sqrt(-2*log(sizeSquared)/sizeSquared)
    return result


In [98]:
%%time
NumberOfPaths = 2000000
callPrice = SimpleMonteCarlo1(0.5,
                     40,
                     42,
                     0.2,
                     0.1,
                     NumberOfPaths)
print(f'Call price: {round(callPrice,2)}, for {NumberOfPaths} paths')

Call price: 4.77, for 2000000 paths
Wall time: 4.3 s


As we can see our routine goes twice as fast as the original, being much more robust and easily configurable thanks to design patterns. We have finally achieved our objective: a fast, efficient and easy to modify MonteCarlo Simulatior for many different options.